In [1]:
from datetime import datetime
from web3 import Web3

In [2]:
# The dafault RPC from ethersjs, change it if it doesn't work: https://infura.io/docs
RPC_Endpoint = 'https://mainnet.infura.io/v3/84842078b09946638c03157f83405213'

w3_eth = Web3(Web3.HTTPProvider(RPC_Endpoint, request_kwargs={'timeout': 20}))
print ('Ethereum connected:', w3_eth.isConnected())

Ethereum connected: True


In [3]:
#https://docs.lido.fi/contracts/lido-oracle
LidoFee = 0.1
LidoOracle = Web3.toChecksumAddress('0x442af784A788A5bd6F42A01Ebe9F287a871243fb')
LidoOracle_abi = """[{"constant":true,
                    "inputs":[],
                    "name":"getLastCompletedReportDelta",
                    "outputs":[{"name":"postTotalPooledEther","type":"uint256"},
                               {"name":"preTotalPooledEther","type":"uint256"},
                               {"name":"timeElapsed","type":"uint256"}],
                    "payable":false,
                    "stateMutability":"view",
                    "type":"function"}
                    ]
                """
getLastCompletedReportDelta = w3_eth.eth.contract(address=LidoOracle, abi=LidoOracle_abi).functions.getLastCompletedReportDelta().call()

postTotalPooledEther = getLastCompletedReportDelta[0]
preTotalPooledEther  = getLastCompletedReportDelta[1]
timeElapsed          = getLastCompletedReportDelta[2]

ThisYear             = datetime.now().year                                
if ThisYear%4==0: #Leap year
    secondsInYear = 60*60*24*366
else:
    secondsInYear = 60*60*24*365

Lido_APR = (postTotalPooledEther - preTotalPooledEther) * secondsInYear / (preTotalPooledEther * timeElapsed)
Lido_APR = Lido_APR*(1-LidoFee)
print("APR minus Lido fee =", Lido_APR)

APR minus Lido fee = 0.03894659476974834
